In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
# 変数urlにSUUMOホームページのURLを格納
url = "https://suumo.jp/chintai/tokyo/sc_shinjuku/?page={}"

# アクセスするためのURLをtarget_urlに格納する
target_url = url.format(1)

print(target_url)

# target_urlへのアクセス結果を変数rに格納
r = requests.get(target_url)

# 取得結果を解析してBeautifulSoupに格納
soup = BeautifulSoup(r.text)

contents = soup.find_all('div', class_='cassetteitem')

https://suumo.jp/chintai/tokyo/sc_shinjuku/?page=1


In [3]:
page = soup.select_one('ol.pagination-parts li:last-child a').text
page_max = int(page)

d_list = []
contents = soup.find_all('div', class_='cassetteitem')

for page_number in range(1, page_max):
    page_url = url.format(page_number)
    page_response = requests.get(page_url)

    for content in contents:
        detail = content.find('div', class_='cassetteitem-detail')
        table = content.find('table', class_='cassetteitem_other')

        title = detail.find('div', class_='cassetteitem_content-title').text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000','')
        address = detail.find('li', class_='cassetteitem_detail-col1').text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000','')
        access = detail.find('li', class_='cassetteitem_detail-col2').text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000','')
        age = detail.find('li', class_='cassetteitem_detail-col3').text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000',''),

        tr_tags = table.find_all('tr', class_='js-cassette_link')
        
        for tr_tag in tr_tags:
            floor, price, first_fee, capacity = tr_tag.find_all('td')[2:6]
            fee, management_fee = price.find_all('li')
            deposit, gratuity = first_fee.find_all('li')
            madori, menseki = capacity.find_all('li')
            
            d =  {
                'title': title,
                'address': address,
                'access': access,
                'age': age,
                'floor': floor.text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000',''),
                'fee': fee.text,
                'management_fee': management_fee.text,
                'deposit': deposit.text,
                'gratuity': gratuity.text,
                'madori': madori.text,
                'menseki': menseki.text
            }

            d_list.append(d)

In [4]:
# pandasでデータフレームに入れて、csvで出力
import pandas as pd
df = pd.DataFrame(d_list)
df.to_csv('house_list.csv')